Testing cleanup of duplicates with massive ammounts of NaN/NaT values

In [3]:
import os
import pandas as pd
from datetime import datetime
import numpy as np

pd.options.mode.copy_on_write = True


def compile_excel_files(folder_path, output_file):



    # Get a list of all Excel files in the folder
    excel_files = [file for file in os.listdir(folder_path) if file.endswith(('.xls', '.xlsx'))]

    # Check if there are any Excel files in the folder
    if not excel_files:
        print("No Excel files found in the specified folder.")
        return

    # Initialize an empty list to store DataFrames
    dfs = []

    # Loop through each Excel file and append its data to the list
    for excel_file in excel_files:
        file_path = os.path.join(folder_path, excel_file)
        df = pd.read_excel(file_path)
        dfs.append(df)

    # Concatenate the list of DataFrames into one
    compiled_data = pd.concat(dfs, ignore_index=True)

    # Convert date columns to datetime dtype
    date_columns = ['Created Date', 'Last Updated Date', 'Payment Date', 'Date Received']
    #clean_data[date_columns] = clean_data[date_columns].apply(lambda x: pd.to_datetime(x, format='%m/%d/%y'))
    compiled_data[date_columns] = compiled_data[date_columns].apply(lambda x: pd.to_datetime(x, format='%m/%d/%y'))

    #clean_data = clean_data.loc[:, date_columns].apply(lambda x: pd.to_datetime(x, format='%m/%d/%y'))

    # Function to find the most recent date among three columns
    def find_latest_date(row):
        return max(row['Created Date'], row['Last Updated Date'], row['Payment Date'], row['Date Received'])

    # Apply the function row-wise to find the latest date
    compiled_data['Latest Record Date'] = compiled_data.apply(find_latest_date, axis=1)
    #clean_data['Latest Record Date'] = clean_data.loc[:, date_columns].apply(find_latest_date, axis=1)
    
    compiled_data_mask = pd.notna(compiled_data['Created Date'])

    compiled_data.loc[compiled_data_mask] = compiled_data.loc[compiled_data_mask].fillna('I am Blank')

    compiled_data.loc[compiled_data_mask] = compiled_data.loc[compiled_data_mask].drop_duplicates()

    nan_values = compiled_data.isna().sum()
    print("NaN values in all columns:")
    print(nan_values)

    compiled_data.loc[compiled_data_mask] = compiled_data.loc[compiled_data_mask].replace('I am Blank', np.nan)

    # # Write the compiled data to a new CSV file
    # compiled_data.loc[compiled_data_mask] = compiled_data.loc[compiled_data_mask].to_csv(output_file, index=False)
    # print(f"Compiled data saved to {output_file}")

    compiled_data.loc[compiled_data_mask] = compiled_data.loc[compiled_data_mask].sort_values(by=['Created Date'], ascending=False, na_position='last')

    compiled_data.loc[compiled_data_mask] = compiled_data.loc[compiled_data_mask].to_csv(output_file, index=False)
    print(f"Compiled data saved to {output_file}")

# Example usage
folder_path = 'C:/Users/MatthewHieger/Documents/My Tableau Repository/Datasources/Coupa Datamart/DATAMART COMPILE TEST/'
output_file = 'C:/Users/MatthewHieger/Documents/My Tableau Repository/Datasources/Coupa Datamart/DATAMART COMPILE TEST/Invoice DMtest Source.csv'

compile_excel_files(folder_path, output_file)

C:\Users\MatthewHieger\AppData\Local\Temp\ipykernel_19232\180179007.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0                 I am Blank
1                 I am Blank
2                 I am Blank
3                 I am Blank
4                 I am Blank
                ...         
32480    2024-01-25 00:00:00
32481             I am Blank
32482    2024-01-04 00:00:00
32483    2024-01-04 00:00:00
32484    2022-11-07 00:00:00
Name: Payment Date, Length: 32485, dtype: object' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  compiled_data.loc[compiled_data_mask] = compiled_data.loc[compiled_data_mask].fillna('I am Blank')
C:\Users\MatthewHieger\AppData\Local\Temp\ipykernel_19232\180179007.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[11874912.0 11874912.0 11874912.0 ... 7860158.0 

NaN values in all columns:
Invoice #                           2852
Supplier                            2852
Net Due Date                        2852
Total                               2852
Status                              2852
Delivery Method                     2852
Dispute Reason                      2852
Invoice Date                        2852
Invoice ID                          2852
Last Updated Date                   2852
Last Updated By                     2852
Payment Date                        2852
Payment Term                        2852
PO Number                           2852
PO ID                               2852
Requester                           2852
Supplier #                          2852
Chart Of Accounts                   2852
Created By                          2852
Created Date                        2852
Credit Applied                      2852
Current Approver                    2852
Date Received                       2852
Exported?                     